In [1]:
import pandas as pd
import numpy as np

# set seed for reproducibility
np.random.seed(0) 

In [2]:
folderpath = "/opt/helthcare-final-project-autism/notebooks/data/processed"
filepath = f"{folderpath}/2017-2018_NSCH_DRC.csv.gzip"

In [3]:
df = pd.read_csv(filepath, index_col='HHID', compression='gzip')
have_autism = df[['K2Q35B']].fillna(2)
df = df[[col for col in df.columns.values if col != 'K2Q35B']]

In [4]:
df_variables = pd.read_csv(f"{folderpath}/2017-2018_description_columns.csv", index_col="column")

In [5]:
description_itens_list = ['screener', 'asd', 'autism']
autism_related_columns = df_variables[df_variables.description.fillna("").str.contains(f"({'|'.join(description_itens_list)})", case=False)].index.tolist()

/opt/conda/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [6]:
df = df[[col for col in df.columns.values if col not in autism_related_columns]]

In [8]:
from scipy.stats import spearmanr
cor_col = []
p_value_col = []
for col in df.columns.values:
    cor, p_value = spearmanr(df[col].values, have_autism.values, nan_policy='omit')
    cor_col.append(cor)
    p_value_col.append(cor)
#     print(f"{col} - spearman: {cor} - p-value: {p_value}")

df_spearman_corr = pd.DataFrame({'spearman': cor_col,
                                 'p_value': p_value_col},
                                 index=df.columns.values)
df_spearman_corr = df_spearman_corr.sort_values(by=['spearman'], ascending=False)

df_spearman_corr = df_spearman_corr.join(df_variables)

filter_func = lambda x, lim: x>lim or x<-lim

selected_columns = (df_spearman_corr[df_spearman_corr.spearman.apply(lambda x: filter_func(x, 0.1))].index.values.tolist())

df_to_model = df[selected_columns]
df_to_model.head()

,K2Q36A,ERRANDALONE,DiffErrand_1718,SC_K2Q16,SpEducPln_1718,SpecServ_1718,SC_K2Q19,SC_K2Q22,K6Q15,ANYOTHER,...,MAKEFRIEND,MakeFr3to5_1718,SpSerAge_1718,learning_1718,SpEdAge_1718,LearnSev_1718,OthrMent_1718,OthMHSev_1718,DevDelay_1718,DevDelSev_1718
HHID,,,,,,,,,,,,,,,,,,,,,
17000010,2.0,NaN,NaN,2.0,NaN,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17000013,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
17000025,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,...,1.0,NaN,0.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0
17000031,2.0,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
17000034,2.0,NaN,NaN,2.0,NaN,2.0,2.0,2.0,NaN,2.0,...,1.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0


In [9]:
y = have_autism.values
X = df_to_model.fillna('999').values

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [15]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.44      0.48      0.46       404
         2.0       0.99      0.98      0.99     15235

    accuracy                           0.97     15639
   macro avg       0.71      0.73      0.72     15639
weighted avg       0.97      0.97      0.97     15639



In [17]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

selector = SelectFromModel(estimator=clf, threshold=0.005).fit(X_train, y_train)
# selector.transform(X_train).shape
logisticRegression = LogisticRegression().fit(selector.transform(X_train), y_train)

print(classification_report(y_test, logisticRegression.predict(selector.transform(X_test))))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

         1.0       0.65      0.34      0.45       404
         2.0       0.98      1.00      0.99     15235

    accuracy                           0.98     15639
   macro avg       0.82      0.67      0.72     15639
weighted avg       0.97      0.98      0.98     15639



In [18]:
from sklearn.ensemble import RandomForestClassifier

randomForestClassifier = RandomForestClassifier(max_depth=3, random_state=0)

randomForestClassifier.fit(X_train, y_train)

print(classification_report(y_test, randomForestClassifier.predict(X_test)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


              precision    recall  f1-score   support

         1.0       0.80      0.03      0.06       404
         2.0       0.97      1.00      0.99     15235

    accuracy                           0.97     15639
   macro avg       0.89      0.51      0.52     15639
weighted avg       0.97      0.97      0.96     15639



In [19]:
from sklearn import svm

svmClassifier = svm.SVC(random_state=0)

svmClassifier.fit(X_train, y_train)

print(classification_report(y_test, svmClassifier.predict(X_test)))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

         1.0       0.73      0.22      0.34       404
         2.0       0.98      1.00      0.99     15235

    accuracy                           0.98     15639
   macro avg       0.85      0.61      0.66     15639
weighted avg       0.97      0.98      0.97     15639



In [39]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report

clf_pipeline = Pipeline([
  ('feature_selection', SelectFromModel(RandomForestClassifier(max_depth=3, random_state=0),
                                        threshold="0.667*median", max_features=25)),
  ('classification', svm.SVC(random_state=0))
])
clf_pipeline.fit(X_train, y_train)

print(classification_report(y_test, clf_pipeline.predict(X_test)))

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_from_model.py:222: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

         1.0       0.68      0.31      0.43       404
         2.0       0.98      1.00      0.99     15235

    accuracy                           0.98     15639
   macro avg       0.83      0.66      0.71     15639
weighted avg       0.97      0.98      0.97     15639



In [40]:
columns_of_model = [y for x, y in zip(clf_pipeline['feature_selection'].get_support(), selected_columns) if x]
print(len(columns_of_model))
columns_of_model

25


['K2Q36A',
 'SC_K2Q16',
 'SpEducPln_1718',
 'SpecServ_1718',
 'SC_K2Q19',
 'SC_K2Q22',
 'ANYOTHER',
 'K2Q30A',
 'mhneeds_1718',
 'SC_K2Q20',
 'CSHCNComplex_1718',
 'K2Q37A',
 'K4Q38',
 'K2Q36B',
 'SESCURRSVC',
 'SC_K2Q21',
 'SC_K2Q23',
 'SC_K2Q14',
 'SC_K2Q17',
 'ANYOTHER_CURR',
 'SESPLANYR',
 'K2Q36C',
 'CondCnt27_1718',
 'MAKEFRIEND',
 'SpSerAge_1718']